In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets.mnist import MNIST
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from torch import optim
import time
from tqdm import tqdm
import os
from torch.utils.data import Dataset
from PIL import Image

In [20]:
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1" 

In [4]:
train_path = '/home/ielab/dataset/ms1m_dataset/train_masked/'
test_path = '/home/ielab/dataset/ms1m_dataset/test_masked'

In [5]:
col_list = os.listdir(train_path)

lb = {string : i for i,string in enumerate(col_list)}

In [6]:
col_list

['1977',
 '2782',
 '1236',
 '2546',
 '1867',
 '1647',
 '1458',
 '1686',
 '2647',
 '1504',
 '1148',
 '2373',
 '2246',
 '3262',
 '407',
 '1569',
 '226',
 '1663',
 '3590',
 '172',
 '1991',
 '192',
 '3183',
 '183',
 '1170',
 '1191',
 '256',
 '1618',
 '1731',
 '2076',
 '203',
 '3264',
 '1791',
 '2709',
 '2362',
 '2053',
 '2657',
 '3002',
 '1525',
 '2954',
 '2820',
 '2903',
 '3372',
 '2719',
 '1141',
 '2222',
 '2730',
 '265',
 '348',
 '1944',
 '3036',
 '1443',
 '1039',
 '3522',
 '1414',
 '1425',
 '1473',
 '1697',
 '2724',
 '1902',
 '1355',
 '2385',
 '466',
 '2276',
 '1015',
 '3338',
 '1481',
 '1433',
 '2144',
 '3578',
 '2529',
 '1486',
 '2157',
 '3501',
 '1259',
 '3095',
 '2240',
 '2883',
 '1915',
 '1004',
 '1107',
 '472',
 '2467',
 '1490',
 '2391',
 '148',
 '3345',
 '2387',
 '413',
 '2111',
 '2326',
 '2694',
 '2840',
 '1285',
 '2279',
 '1658',
 '3350',
 '3524',
 '1442',
 '1570',
 '2428',
 '2079',
 '2088',
 '1153',
 '2571',
 '3541',
 '1718',
 '2159',
 '2539',
 '2744',
 '2244',
 '1319',
 '336

In [7]:
lb

{'1977': 0,
 '2782': 1,
 '1236': 2,
 '2546': 3,
 '1867': 4,
 '1647': 5,
 '1458': 6,
 '1686': 7,
 '2647': 8,
 '1504': 9,
 '1148': 10,
 '2373': 11,
 '2246': 12,
 '3262': 13,
 '407': 14,
 '1569': 15,
 '226': 16,
 '1663': 17,
 '3590': 18,
 '172': 19,
 '1991': 20,
 '192': 21,
 '3183': 22,
 '183': 23,
 '1170': 24,
 '1191': 25,
 '256': 26,
 '1618': 27,
 '1731': 28,
 '2076': 29,
 '203': 30,
 '3264': 31,
 '1791': 32,
 '2709': 33,
 '2362': 34,
 '2053': 35,
 '2657': 36,
 '3002': 37,
 '1525': 38,
 '2954': 39,
 '2820': 40,
 '2903': 41,
 '3372': 42,
 '2719': 43,
 '1141': 44,
 '2222': 45,
 '2730': 46,
 '265': 47,
 '348': 48,
 '1944': 49,
 '3036': 50,
 '1443': 51,
 '1039': 52,
 '3522': 53,
 '1414': 54,
 '1425': 55,
 '1473': 56,
 '1697': 57,
 '2724': 58,
 '1902': 59,
 '1355': 60,
 '2385': 61,
 '466': 62,
 '2276': 63,
 '1015': 64,
 '3338': 65,
 '1481': 66,
 '1433': 67,
 '2144': 68,
 '3578': 69,
 '2529': 70,
 '1486': 71,
 '2157': 72,
 '3501': 73,
 '1259': 74,
 '3095': 75,
 '2240': 76,
 '2883': 77,
 '1915

In [8]:
batch_size = 64

In [9]:
num_classe = len(col_list)

In [10]:
trans = transforms.Compose([
    transforms.Resize((112,112)),
    transforms.ToTensor(),
    ])

In [11]:
class CustomDataset(Dataset):
    def __init__(self, img_dir, transform=None, target_transform=None):
        self.img_dir = img_dir
        self.dir_list = os.listdir(img_dir)
        self.transform = transform
        self.target_transform = target_transform

        self.files = []
        for folder in self.dir_list:
          folder_list = os.listdir(os.path.join(img_dir, folder))
          for file in folder_list:
            self.files.append([
                os.path.join(self.img_dir, folder+'/'+file),
                folder])

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        image = Image.open(self.files[idx][0])
        image = image.convert('RGB')
        image = self.transform(image)

        label = self.files[idx][1]
        label = lb[label]

        return image, label

In [12]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

trainset = CustomDataset(train_path, trans)
trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=4)

testset = CustomDataset(test_path, trans)
testloader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=4)

In [13]:
class ArcFaceLoss(nn.Module):
    def __init__(self, num_classes, embedding_size, margin, scale):
        """
        ArcFace: Additive Angular Margin Loss for Deep Face Recognition
        (https://arxiv.org/pdf/1801.07698.pdf)
        Args:
            num_classes: The number of classes in your training dataset
            embedding_size: The size of the embeddings that you pass into
            margin: m in the paper, the angular margin penalty in radians
            scale: s in the paper, feature scale
        """
        super().__init__()
        self.num_classes = num_classes
        self.embedding_size = embedding_size
        self.margin = margin
        self.scale = scale
        
        self.W = torch.nn.Parameter(torch.Tensor(num_classes, embedding_size))
        nn.init.xavier_normal_(self.W)
        
    def forward(self, embeddings, labels):
        """
        Args:
            embeddings: (None, embedding_size)
            labels: (None,)
        Returns:
            loss: scalar
        """
        cosine = self.get_cosine(embeddings) # (None, n_classes)
        mask = self.get_target_mask(labels) # (None, n_classes)
        cosine_of_target_classes = cosine[mask == 1] # (None, )
        modified_cosine_of_target_classes = self.modify_cosine_of_target_classes(
            cosine_of_target_classes
        ) # (None, )
        diff = (modified_cosine_of_target_classes - cosine_of_target_classes).unsqueeze(1) # (None,1)
        logits = cosine + (mask * diff) # (None, n_classes)
        logits = self.scale_logits(logits) # (None, n_classes)
        return logits
        
    def get_cosine(self, embeddings):
        """
        Args:
            embeddings: (None, embedding_size)
        Returns:
            cosine: (None, n_classes)
        """
        cosine = F.linear(F.normalize(embeddings), F.normalize(self.W))
        return cosine
    
    def get_target_mask(self, labels):
        """
        Args:
            labels: (None,)
        Returns:
            mask: (None, n_classes)
        """
        batch_size = labels.size(0)
        onehot = torch.zeros(batch_size, self.num_classes, device=labels.device)
        onehot.scatter_(1, labels.unsqueeze(-1), 1)
        return onehot
        
    def modify_cosine_of_target_classes(self, cosine_of_target_classes):
        """
        Args:
            cosine_of_target_classes: (None,)
        Returns:
            modified_cosine_of_target_classes: (None,)
        """
        eps = 1e-6
        # theta in the paper
        angles = torch.acos(torch.clamp(cosine_of_target_classes, -1 + eps, 1 - eps))
        return torch.cos(angles + self.margin)
    
    def scale_logits(self, logits):
        """
        Args:
            logits: (None, n_classes)
        Returns:
            scaled_logits: (None, n_classes)
        """
        return logits * self.scale
    
class SoftmaxLoss(nn.Module):
    def __init__(self, num_classes, embedding_size):
        """
        Regular softmax loss (1 fc layer without bias + CrossEntropyLoss)
        Args:
            num_classes: The number of classes in your training dataset
            embedding_size: The size of the embeddings that you pass into
        """
        super().__init__()
        self.num_classes = num_classes
        self.embedding_size = embedding_size
        
        self.W = torch.nn.Parameter(torch.Tensor(num_classes, embedding_size))
        nn.init.xavier_normal_(self.W)
        
    def forward(self, embeddings, labels):
        """
        Args:
            embeddings: (None, embedding_size)
            labels: (None,)
        Returns:
            loss: scalar
        """
        logits = F.linear(embeddings, self.W)
        return nn.CrossEntropyLoss()(logits, labels)

In [14]:
class Embedder(nn.Module):
    def __init__(self, embedding_size):
        super().__init__()
        self.model = torchvision.models.resnet50(pretrained = False)
        self.dropout = nn.Dropout(0.5)
        
        self.classifier = nn.Linear(1000, embedding_size)

    def forward(self, images):
        outputs = self.model(images)
        outputs = self.dropout(outputs)
        outputs = self.classifier(outputs)
        return outputs

In [15]:
embedding_size = 512
max_epochs = 100


embedder = Embedder(embedding_size=embedding_size)
arcface = ArcFaceLoss(num_classes=num_classe, embedding_size=embedding_size,margin=0.3, scale=30.0)

if torch.cuda.device_count() > 1:
    embedder = nn.DataParallel(embedder)
    arcface = nn.DataParallel(arcface)
embedder = embedder.to(device)
arcface = arcface.to(device)

optimizer = optim.Adam(embedder.parameters(), lr=1e-3 ) 
criterion = nn.CrossEntropyLoss()
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 5, gamma = 0.9)


start = time.time()
for epoch in range(max_epochs):
    e_time = time.time()
    for i, (images, labels) in enumerate(trainloader):
        embedder.train()
        images = images.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        embeddings = embedder(images)
        
        logits = arcface(embeddings, labels)
        loss = criterion(logits, labels)
        loss.backward() 
        optimizer.step() 
        
        if (i+1) % 64 == 0:
            print(f'Epoch: {epoch} - Batch: {i*batch_size} - Loss: {loss:.6f} - Time:{time.time() - e_time}')
            with open('/home/ielab/project/samples/masked_loss.txt', 'a') as file:
                file.write(f'{loss:.6f}\n')
            

/home/ielab/anaconda3/envs/arcface/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/home/ielab/anaconda3/envs/arcface/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Epoch: 0 - Batch: 4032 - Loss: 17.361507 - Time:12.214573621749878
Epoch: 0 - Batch: 8128 - Loss: 17.051888 - Time:21.63335609436035
Epoch: 0 - Batch: 12224 - Loss: 17.179497 - Time:31.055153846740723
Epoch: 0 - Batch: 16320 - Loss: 17.309452 - Time:40.4832181930542
Epoch: 0 - Batch: 20416 - Loss: 17.119143 - Time:49.915668964385986
Epoch: 0 - Batch: 24512 - Loss: 17.253811 - Time:59.37677884101868
Epoch: 0 - Batch: 28608 - Loss: 17.187641 - Time:68.81080675125122
Epoch: 0 - Batch: 32704 - Loss: 16.971745 - Time:78.2370913028717
Epoch: 0 - Batch: 36800 - Loss: 16.974154 - Time:87.66201162338257
Epoch: 0 - Batch: 40896 - Loss: 16.931511 - Time:97.10685849189758
Epoch: 0 - Batch: 44992 - Loss: 17.151213 - Time:106.5352783203125
Epoch: 0 - Batch: 49088 - Loss: 17.150532 - Time:115.96126341819763
Epoch: 0 - Batch: 53184 - Loss: 16.965466 - Time:125.39626431465149
Epoch: 0 - Batch: 57280 - Loss: 17.024460 - Time:134.81561160087585
Epoch: 0 - Batch: 61376 - Loss: 17.164036 - Time:144.2313110

Epoch: 3 - Batch: 49088 - Loss: 16.591591 - Time:113.18501925468445
Epoch: 3 - Batch: 53184 - Loss: 16.338840 - Time:122.59879493713379
Epoch: 3 - Batch: 57280 - Loss: 16.337900 - Time:132.01120615005493
Epoch: 3 - Batch: 61376 - Loss: 16.325525 - Time:141.423259973526
Epoch: 3 - Batch: 65472 - Loss: 16.344061 - Time:150.853586435318
Epoch: 3 - Batch: 69568 - Loss: 16.366693 - Time:160.26542234420776
Epoch: 3 - Batch: 73664 - Loss: 16.547134 - Time:169.67703485488892
Epoch: 3 - Batch: 77760 - Loss: 16.369228 - Time:179.0916759967804
Epoch: 3 - Batch: 81856 - Loss: 16.378080 - Time:188.52169489860535
Epoch: 3 - Batch: 85952 - Loss: 16.414351 - Time:197.9334876537323
Epoch: 3 - Batch: 90048 - Loss: 16.586966 - Time:207.34606552124023
Epoch: 3 - Batch: 94144 - Loss: 16.494598 - Time:216.77755498886108
Epoch: 3 - Batch: 98240 - Loss: 16.475344 - Time:226.18973350524902
Epoch: 3 - Batch: 102336 - Loss: 16.330709 - Time:235.60386109352112
Epoch: 3 - Batch: 106432 - Loss: 16.413763 - Time:245

Epoch: 6 - Batch: 94144 - Loss: 15.033865 - Time:216.70112824440002
Epoch: 6 - Batch: 98240 - Loss: 15.388899 - Time:226.13171672821045
Epoch: 6 - Batch: 102336 - Loss: 15.572194 - Time:235.54218554496765
Epoch: 6 - Batch: 106432 - Loss: 15.699821 - Time:244.9537239074707
Epoch: 6 - Batch: 110528 - Loss: 15.215359 - Time:254.36501479148865
Epoch: 6 - Batch: 114624 - Loss: 15.226972 - Time:263.77609491348267
Epoch: 6 - Batch: 118720 - Loss: 15.408610 - Time:273.2069489955902
Epoch: 6 - Batch: 122816 - Loss: 15.197136 - Time:282.6181755065918
Epoch: 6 - Batch: 126912 - Loss: 15.423554 - Time:292.03005480766296
Epoch: 6 - Batch: 131008 - Loss: 15.627878 - Time:301.4608783721924
Epoch: 6 - Batch: 135104 - Loss: 15.104167 - Time:310.87580704689026
Epoch: 6 - Batch: 139200 - Loss: 14.933500 - Time:320.29018092155457
Epoch: 6 - Batch: 143296 - Loss: 15.294626 - Time:329.7031362056732
Epoch: 6 - Batch: 147392 - Loss: 15.418038 - Time:339.1132085323334
Epoch: 6 - Batch: 151488 - Loss: 15.009151

Epoch: 9 - Batch: 135104 - Loss: 11.174631 - Time:310.91861963272095
Epoch: 9 - Batch: 139200 - Loss: 10.601727 - Time:320.3342695236206
Epoch: 9 - Batch: 143296 - Loss: 10.448202 - Time:329.74677181243896
Epoch: 9 - Batch: 147392 - Loss: 10.458442 - Time:339.1777720451355
Epoch: 9 - Batch: 151488 - Loss: 10.865146 - Time:348.59156346321106
Epoch: 10 - Batch: 4032 - Loss: 10.511045 - Time:9.577880144119263
Epoch: 10 - Batch: 8128 - Loss: 11.571114 - Time:18.99225616455078
Epoch: 10 - Batch: 12224 - Loss: 10.537813 - Time:28.43175768852234
Epoch: 10 - Batch: 16320 - Loss: 11.127252 - Time:37.844266176223755
Epoch: 10 - Batch: 20416 - Loss: 10.165695 - Time:47.25762915611267
Epoch: 10 - Batch: 24512 - Loss: 11.201471 - Time:56.66965174674988
Epoch: 10 - Batch: 28608 - Loss: 10.563341 - Time:66.08358430862427
Epoch: 10 - Batch: 32704 - Loss: 10.469930 - Time:75.49524188041687
Epoch: 10 - Batch: 36800 - Loss: 11.213416 - Time:84.90872001647949
Epoch: 10 - Batch: 40896 - Loss: 9.760038 - Ti

...

Epoch: 85 - Batch: 44992 - Loss: 0.069632 - Time:103.75098085403442
Epoch: 85 - Batch: 49088 - Loss: 0.121636 - Time:113.16314244270325
Epoch: 85 - Batch: 53184 - Loss: 0.088122 - Time:122.57370281219482
Epoch: 85 - Batch: 57280 - Loss: 0.050035 - Time:131.98789405822754
Epoch: 85 - Batch: 61376 - Loss: 0.170449 - Time:141.40217542648315
Epoch: 85 - Batch: 65472 - Loss: 0.051108 - Time:150.8343164920807
Epoch: 85 - Batch: 69568 - Loss: 0.084532 - Time:160.25080585479736
Epoch: 85 - Batch: 73664 - Loss: 0.244406 - Time:169.68177485466003
Epoch: 85 - Batch: 77760 - Loss: 0.067865 - Time:179.09636545181274
Epoch: 85 - Batch: 81856 - Loss: 0.238015 - Time:188.51282405853271
Epoch: 85 - Batch: 85952 - Loss: 0.027053 - Time:197.92763090133667
Epoch: 85 - Batch: 90048 - Loss: 0.065620 - Time:207.3419852256775
Epoch: 85 - Batch: 94144 - Loss: 0.355328 - Time:216.7568907737732
Epoch: 85 - Batch: 98240 - Loss: 0.342298 - Time:226.17180681228638
Epoch: 85 - Batch: 102336 - Loss: 0.222909 - Time:2

Epoch: 88 - Batch: 85952 - Loss: 0.140047 - Time:197.9274034500122
Epoch: 88 - Batch: 90048 - Loss: 0.157891 - Time:207.35666918754578
Epoch: 88 - Batch: 94144 - Loss: 0.133871 - Time:216.771639585495
Epoch: 88 - Batch: 98240 - Loss: 0.028359 - Time:226.18631148338318
Epoch: 88 - Batch: 102336 - Loss: 0.158983 - Time:235.61828112602234
Epoch: 88 - Batch: 106432 - Loss: 0.269634 - Time:245.03595566749573
Epoch: 88 - Batch: 110528 - Loss: 0.153457 - Time:254.45086240768433
Epoch: 88 - Batch: 114624 - Loss: 0.177849 - Time:263.864999294281
Epoch: 88 - Batch: 118720 - Loss: 0.150990 - Time:273.27823519706726
Epoch: 88 - Batch: 122816 - Loss: 0.122434 - Time:282.6911692619324
Epoch: 88 - Batch: 126912 - Loss: 0.087408 - Time:292.10709404945374
Epoch: 88 - Batch: 131008 - Loss: 0.125564 - Time:301.5213973522186
Epoch: 88 - Batch: 135104 - Loss: 0.094853 - Time:310.95368552207947
Epoch: 88 - Batch: 139200 - Loss: 0.038266 - Time:320.36978244781494
Epoch: 88 - Batch: 143296 - Loss: 0.169200 - 

Epoch: 91 - Batch: 126912 - Loss: 0.045398 - Time:292.14849519729614
Epoch: 91 - Batch: 131008 - Loss: 0.136917 - Time:301.5810775756836
Epoch: 91 - Batch: 135104 - Loss: 0.329729 - Time:310.9952304363251
Epoch: 91 - Batch: 139200 - Loss: 0.027150 - Time:320.4084973335266
Epoch: 91 - Batch: 143296 - Loss: 0.095013 - Time:329.82299733161926
Epoch: 91 - Batch: 147392 - Loss: 0.199599 - Time:339.23641443252563
Epoch: 91 - Batch: 151488 - Loss: 0.026404 - Time:348.65218138694763
Epoch: 92 - Batch: 4032 - Loss: 0.030563 - Time:9.57466173171997
Epoch: 92 - Batch: 8128 - Loss: 0.102818 - Time:18.99150037765503
Epoch: 92 - Batch: 12224 - Loss: 0.042228 - Time:28.43429732322693
Epoch: 92 - Batch: 16320 - Loss: 0.058534 - Time:37.84827399253845
Epoch: 92 - Batch: 20416 - Loss: 0.213977 - Time:47.26348829269409
Epoch: 92 - Batch: 24512 - Loss: 0.094274 - Time:56.67705249786377
Epoch: 92 - Batch: 28608 - Loss: 0.034899 - Time:66.1086573600769
Epoch: 92 - Batch: 32704 - Loss: 0.022472 - Time:75.524

Epoch: 95 - Batch: 20416 - Loss: 0.045294 - Time:47.27451682090759
Epoch: 95 - Batch: 24512 - Loss: 0.279192 - Time:56.690237045288086
Epoch: 95 - Batch: 28608 - Loss: 0.077461 - Time:66.10448789596558
Epoch: 95 - Batch: 32704 - Loss: 0.079831 - Time:75.52002048492432
Epoch: 95 - Batch: 36800 - Loss: 0.049170 - Time:84.95032405853271
Epoch: 95 - Batch: 40896 - Loss: 0.035838 - Time:94.36286067962646
Epoch: 95 - Batch: 44992 - Loss: 0.037414 - Time:103.77632284164429
Epoch: 95 - Batch: 49088 - Loss: 0.050429 - Time:113.18846726417542
Epoch: 95 - Batch: 53184 - Loss: 0.246711 - Time:122.60201644897461
Epoch: 95 - Batch: 57280 - Loss: 0.053213 - Time:132.03136658668518
Epoch: 95 - Batch: 61376 - Loss: 0.061005 - Time:141.44597744941711
Epoch: 95 - Batch: 65472 - Loss: 0.027268 - Time:150.85914182662964
Epoch: 95 - Batch: 69568 - Loss: 0.146969 - Time:160.29057908058167
Epoch: 95 - Batch: 73664 - Loss: 0.086414 - Time:169.7052915096283
Epoch: 95 - Batch: 77760 - Loss: 0.032095 - Time:179.1

Epoch: 98 - Batch: 61376 - Loss: 0.106359 - Time:141.43098378181458
Epoch: 98 - Batch: 65472 - Loss: 0.093846 - Time:150.84435391426086
Epoch: 98 - Batch: 69568 - Loss: 0.029850 - Time:160.2571566104889
Epoch: 98 - Batch: 73664 - Loss: 0.095764 - Time:169.68763613700867
Epoch: 98 - Batch: 77760 - Loss: 0.044316 - Time:179.09974813461304
Epoch: 98 - Batch: 81856 - Loss: 0.178024 - Time:188.51147270202637
Epoch: 98 - Batch: 85952 - Loss: 0.037120 - Time:197.92380690574646
Epoch: 98 - Batch: 90048 - Loss: 0.059585 - Time:207.336275100708
Epoch: 98 - Batch: 94144 - Loss: 0.037634 - Time:216.74654579162598
Epoch: 98 - Batch: 98240 - Loss: 0.095407 - Time:226.15892505645752
Epoch: 98 - Batch: 102336 - Loss: 0.074807 - Time:235.57213878631592
Epoch: 98 - Batch: 106432 - Loss: 0.025861 - Time:245.0008761882782
Epoch: 98 - Batch: 110528 - Loss: 0.136065 - Time:254.41507983207703
Epoch: 98 - Batch: 114624 - Loss: 0.082256 - Time:263.8270146846771
Epoch: 98 - Batch: 118720 - Loss: 0.170119 - Time

In [18]:
embedder = torch.load("masked_model.pth", map_location=device)
print(embedder)

DataParallel(
  (module): Embedder(
    (model): ResNet(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inpla

In [19]:
#  정확도 측정
train_results = []
train_labels = []
test_results = []
test_labels = []

embedder.eval()
with torch.no_grad():
  for img, label in trainloader:
    img = img.cuda()
    train_results.append(embedder(img).cpu().detach().numpy())
    train_labels.append(label)

train_results = np.concatenate(train_results)
train_labels = np.concatenate(train_labels)

embedder.eval()
with torch.no_grad():
  for img, label in testloader:
    img = img.cuda()
    test_results.append(embedder(img).cpu().detach().numpy())
    test_labels.append(label)

test_results = np.concatenate(test_results)
test_labels = np.concatenate(test_labels)

In [22]:
from sklearn.neighbors import KNeighborsClassifier


# kNN 모델 선언
k = 50
model = KNeighborsClassifier(n_neighbors = k)
# 모델 학습
model.fit(train_results, train_labels)
#knn 검증
train_pred = model.predict(train_results)
train_acc = (train_pred == train_labels).mean()
with open('/home/ielab/project/samples/gan_train_acc.txt', 'a') as file:
    file.write(f'{train_acc:.6f}\n')
print(f'train acc : {train_acc}')
test_pred = model.predict(test_results)
test_acc = (test_pred == test_labels).mean()
with open('/home/ielab/project/samples/gan_test_acc.txt', 'a') as file:
    file.write(f'{test_acc:.6f}\n')
print(f'test acc : {test_acc}')

train acc : 0.96643107249384
test acc : 0.6540348327147195


In [23]:
# F1-Score
f1 = metrics.classification_report(test_labels, test_pred)
with open('/home/ielab/project/samples/gan_test_f1.txt', 'a') as file:
    file.write(f'{f1}\n')

/home/ielab/anaconda3/envs/arcface/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ielab/anaconda3/envs/arcface/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ielab/anaconda3/envs/arcface/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu

In [24]:
print(f1)

              precision    recall  f1-score   support

           0       1.00      0.85      0.92        13
           1       1.00      0.58      0.73        19
           2       0.90      0.86      0.88        21
           3       0.00      0.00      0.00         1
           4       0.36      0.67      0.47         6
           5       0.93      0.93      0.93        15
           6       0.00      0.00      0.00         2
           7       1.00      0.68      0.81        19
           8       1.00      0.67      0.80        24
           9       0.13      0.67      0.22        52
          10       0.88      0.75      0.81        20
          11       1.00      0.71      0.83        17
          12       1.00      0.67      0.80         9
          13       0.58      0.50      0.54        14
          14       0.67      0.20      0.31        10
          15       1.00      0.53      0.69        17
          16       0.12      0.20      0.15         5
          17       1.00    

In [35]:
torch.save(embedder, './masked_model.pth')